# PyTorch Workflow

let's explore an example PyTorch end-to-end workflow.

Resources:
* Ground truth notebook: https://github.com/mrdbourke/pytorch-deep-learning/blob/main/01_pytorch_workflow.ipynb
* Book version of notebook: https://www.learnpytorch.io/01_pytorch_workflow/


In [7]:
what_were_covering = {1: "Data (prepare and load)",
                      2: "Build model",
                      3: "Fitting the model to data (training)",
                      4: "Making predictions and evaluating the model (inference)",
                      5: "Saving and loading a model",
                      6: "Putting it all together"
                      }
what_were_covering

{1: 'Data (prepare and load)',
 2: 'Build model',
 3: 'Fitting the model to data (training)',
 4: 'Making predictions and evaluating the model (inference)',
 5: 'Saving and loading a model',
 6: 'Putting it all together'}

In [8]:
import torch
from torch import nn # nn contains all of PyTotch's building blocks
import matplotlib.pyplot as plt

# Check PyTorch version
print(torch.__version__)

2.3.0


https://pytorch.org/docs/stable/nn.html

## 1. Data (Preparing and Loading)

Data can be almost anything... in machine learning.

* Excel speadsheet
* Images of any kind
* Videos (YouTube has lots of data...)
* Audio like songs or podcasts
* DNA
* Text
* ETC

Machine learning is a game of two parts:
1. Get data in a numerical representation.
2. Build a model to learn patterns in that numerical representation.

To Showcase this, let's create some *known* data using the linear regression formula. 

We'll use a linear regression formula to make a striaght line with *known* **parameters**. 

Parameters= something that a model learns


In [9]:
# Create *known* parameters
weight = 0.7
bias = 0.3

# Create
start = 0
end = 1
step = 0.02
# captial represents a matrix or tensor
X = torch.arange(start, end, step).unsqueeze(dim= 1)
# lowercase represents a vector
y = weight * X + bias

X[:10], y[:10]



(tensor([[0.0000],
         [0.0200],
         [0.0400],
         [0.0600],
         [0.0800],
         [0.1000],
         [0.1200],
         [0.1400],
         [0.1600],
         [0.1800]]),
 tensor([[0.3000],
         [0.3140],
         [0.3280],
         [0.3420],
         [0.3560],
         [0.3700],
         [0.3840],
         [0.3980],
         [0.4120],
         [0.4260]]),
 50,
 50)

In [10]:
len(X), len(y)

(50, 50)